In [134]:
import numpy as np
from numpy import ndarray
import nnfs
from nnfs.datasets import spiral_data

# Basics
### Matrix shapes
The input matrix will have rows that correspond to the number of samples. The columns will correspond to the number of features

In [135]:
inputs = [[1.0, 2.0, 3.0, 2.5],
         [2.0, 5.0, -1.0, 2.0],
         [-1.5, 2.7, 3.3, -0.8]]
inputs = np.array(inputs)
inputs

array([[ 1. ,  2. ,  3. ,  2.5],
       [ 2. ,  5. , -1. ,  2. ],
       [-1.5,  2.7,  3.3, -0.8]])

Weights will have rows that correspond to the number of neurons. The columns will correspond to the number of features in the samples.

In [136]:
weights = [[0.2, 0.8, -0.5, 1.0],
           [0.5, -0.91, 0.26, -0.5],
           [-0.26, -0.27, 0.17, 0.87]]

biases = [2, 3, 0.5]

weights = np.array(weights)
biases = np.array(biases)

### Which parameter to put first?
We have the inputs as first parameter of the matrix product as the result will consist of a list of layer outputs per each sample. 

In [137]:
layer_output = inputs @ weights.T

# Dense Layer Class

A dense layer, also known as a fully-connected layer.

Note: we initalize weigghts as (features, n_neurons) to skip transposing

In [138]:
class Dense:
    """
    A layer of neurons.
    features: number of input features
    n_neurons: number of neurons in a layer
    """
    def __init__(self, features: int, n_neurons: int):
        #Initalize weights and biases
        self.weights = 0.01 * np.random.randn(features, n_neurons)
        self.biases = np.zeros((1, n_neurons))
    
    def forward(self, inputs: ndarray):
        """Calculate output values from inputs, weights and biases"""
        self.output = inputs @ self.weights + self.biases

In [139]:
nnfs.init()

# create data set with 3 classes, 2 input features and 
X, y = spiral_data(samples=100, classes=3)

# testing dense layer with 2 input features and 3 neurons (output values)
dense_1 = Dense(2, 3)

# perform forward pass of training data through the layer
dense_1.forward(X)
print(dense_1.output[:5])

[[ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
 [-1.04751874e-04  1.13953603e-04 -4.79834998e-05]
 [-2.74148420e-04  3.17291502e-04 -8.69217984e-05]
 [-4.21883655e-04  5.26662567e-04 -5.59126856e-05]
 [-5.77076804e-04  7.14014051e-04 -8.94304394e-05]]


# Activation Functions
## Rectified Linear Units (ReLU)

$$f(x)= max(0, x) $$ 

In [140]:
class ActivationReLU:
    """
    Rectified Linear Units activation function
    """
    def forward(self, inputs):
        self.output = np.maximum(0, inputs)

In [141]:
# testing the activation class
activation = ActivationReLU()
activation.forward(dense_1.output)
print(activation.output[:5])

[[0.         0.         0.        ]
 [0.         0.00011395 0.        ]
 [0.         0.00031729 0.        ]
 [0.         0.00052666 0.        ]
 [0.         0.00071401 0.        ]]


## Softmax Activation Function
$$S_{i,j} = \frac{e^{z_{i,j}}}{\sum_{l=1}^L e^{z_{i,l}}}$$
where index $i$ means the current sample and the index $j$ means the current output in the sample

In [142]:
# example to understand softmax
ex_layer_outputs = [4.8, 1.21, 2.385]
# for each value in vector, calculate the exponential value
exp_values = np.exp(ex_layer_outputs)
print(f'exponentiated values: {exp_values}')

# normalize values
norm_values = exp_values / np.sum(exp_values)
print(f'normalized exponential values: {norm_values}')

exponentiated values: [121.51041752   3.35348465  10.85906266]
normalized exponential values: [0.89528266 0.02470831 0.08000903]


To train in batches, we need to convert the function to accept layer outputs in batches

In [143]:
ex_layer_outputs_2 = np.array([[ 4.8 , 1.21 , 2.385 ],
                               [ 8.9 , - 1.81 , 0.2 ],
                               [ 1.41 , 1.051 , 0.026 ]])
ex_layer_outputs_2

# Brief review of np.sum
## sum without axis sums up all the values
print(f'Sum without axis: {np.sum(ex_layer_outputs_2)}')

## sum axis=0 will give u sum of the columns
print(f'Sum without axis: {np.sum(ex_layer_outputs_2, axis=0)}')

## sum axis=1 will give u sum of the rows
print(f'Sum without axis: {np.sum(ex_layer_outputs_2, axis=1)}')

## keep dim will output a single value per sample
print(f'Sum without axis: {np.sum(ex_layer_outputs_2, axis=1, keepdims=True)}')

# exp_values = np.exp(ex_layer_outputs)
# probas =exp_values / np.sum(exp_values, axis=1, keepdims=True)

Sum without axis: 18.172
Sum without axis: [15.11   0.451  2.611]
Sum without axis: [8.395 7.29  2.487]
Sum without axis: [[8.395]
 [7.29 ]
 [2.487]]


## Handiling exploding values
Softmax activation is a source of "exploding" values, due to the nature of exponential functions. In order to prevet overflows, we subtract the maximum value from a list of inputs. This would change the output values to be in a range from some negative value to 0 (max minus itself is 0).

In [144]:
class ActivationSoftmax:
    """
    Softmax activation function
    """
    def forward(self, inputs):
        # get exp values whilst preventing overflow
        exp_values = np.exp(inputs - np.max(inputs, axis=1, keepdims=True))
        
        #normalize for each sample
        probabs = exp_values / np.sum(exp_values, axis=1, keepdims=True)
        
        self.output = probabs

In [145]:
# testing 
X, y = spiral_data(samples=100, classes=3)

# first layer with 3 neurons
dense_1 = Dense(2, 3)

# ReLU activation for first layer
activation_1 = ActivationReLU()

# second layer with 3 input feautures (input features in the second layer correspond to the outputs from the first layer)
# and 3 neurons
dense_2 = Dense(3, 3)

# softmax activagtion for second layer
activation_2 =  ActivationSoftmax()

# forward pass through the network
dense_1.forward(X)
activation_1.forward(dense_1.output) #inputs to the activation function will be the output from the first dense layer
dense_2.forward(activation_1.output)
activation_2.forward(dense_2.output)

print(activation_2.output[:5])

[[0.33333334 0.33333334 0.33333334]
 [0.33333334 0.33333334 0.33333334]
 [0.33333334 0.33333334 0.33333334]
 [0.33333334 0.33333334 0.33333334]
 [0.33333334 0.33333334 0.33333334]]


# Caclulating Network Error with Loss
## Categorical Cross-Entropy Loss
$$L_i = - \sum_j y_i \log(\hat{y_{i, j}})$$
Where $L_i$ denote the sample loss value, 

$i$ is the i-th sample in the set, 

$j$ is the label/output index,

$y$ is the target values 

and $\hat y$ denotes the predicted values.

In [146]:
softmax_outputs = np.array([[ 0.7 , 0.1 , 0.2 ],
                            [ 0.1 , 0.5 , 0.4 ],
                            [ 0.02 , 0.9 , 0.08 ]])
class_tragets = [0, 1, 1]
confidences = softmax_outputs[range(len(softmax_outputs)), class_tragets]
neg_log = -np.log(confidences)
average_loss = np.mean(neg_log)
print(f'list of confidences at the target indices for each sample: {confidences}')
print(f'negative log of the confidences: {neg_log}')
print(f'average loss: {average_loss:.2f}')

list of confidences at the target indices for each sample: [0.7 0.5 0.9]
negative log of the confidences: [0.35667494 0.69314718 0.10536052]
average loss: 0.39


## Specifics of the categorical cross entropy function
The cat-cross entropy function handles targets both as sparse (contain correct class numbers) and as one-hot encoded vectors. The check is performed by counting dimensions - if targets are 1D (like a list) they are sparce, but if they are 2d (like a list of lists), then they are sets of one-hot encoded vectors.

Furthermore, it is possible that the model will have full confidence for one label. It is also possible that the model will assign full confidence to a value that wasn't the target. Calculating the loss of this 0 confidence will give us an error (log(0) is undefined).

To solve this, we add a very small value to the confidences. However, this introduces two new issues. First, when the confidence is 1, the new value will be slighlty larger than one resulting in a negative value instead of being zero - $-np.log(1+1e-7)$. Second, a confidence can be shifted towards 1. To prevent both these issues, we clip values from both sides by the same number. Thus, the lowest possible value will become $1+1e-7$ and the highest possible value will become $1-1e-7$.

In [147]:
class Loss:
    """
    Base loss class that calculatges the data and regularizes losses given model
    output and ground truth values.
    """
    def calculate(self, output:ndarray, y:ndarray) -> float:
        loss = np.mean(self.forward(output, y))
        
        return  loss
        
class CatCrossEntropy(Loss):
    def forward(self, y_pred, y_true):
        # num of samples in a batch
        samples = len(y_pred)
        
        # clip data on both sides
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)
        
        # calculate probabilities for target values
        # if categorical labels
        if len(y_true.shape) == 1:
            confs = y_pred_clipped[range(samples), y_true]
        
        # mask values - if one-hot encoded labels
        elif len(y_true.shape) == 2:
            confs = np.sum(y_pred_clipped * y_true, axis=1)
            
        neg_log_likelihoods = -np.log(confs)
        
        return neg_log_likelihoods

In [148]:
softmax_outputs = np.array([[ 0.7 , 0.1 , 0.2 ],
                            [ 0.1 , 0.5 , 0.4 ],
                            [ 0.02 , 0.9 , 0.08 ]])
targets_sparse = np.array([0, 1, 1])
targets_ohe = np.array([[ 1 , 0 , 0 ],
                        [ 0 , 1 , 0 ],
                        [ 0 , 1 , 0 ]])

loss_func = CatCrossEntropy()
loss_sparse= loss_func.calculate(softmax_outputs, targets_sparse)
loss_ohe = loss_func.calculate(softmax_outputs, targets_ohe)

print(f'Loss spares: {loss_sparse}')
print(f'Loss ohe: {loss_ohe}')

Loss spares: 0.38506088005216804
Loss ohe: 0.38506088005216804


In [149]:
# continuing the testing
# output from the second layer
# activation_2.output
loss = loss_func.calculate(activation_2.output, y)
print(f'Loss: {loss:.2f}')

Loss: 1.10


## Accuracy
In common practive, accuracy is used along with loss. Accuracy describes how often the largest confidence is the correct class in terms of a fraction.

In [150]:
# calculate accuracy from the output of activation_2 and targets
preds = np.argmax(activation_2.output, axis=1)
# if targets are OHE then convert them
if len(y.shape) == 2:
    y = np.argmax(y, axis=1)
accuracy = np.mean(preds == y)
print(f'Accuracy: {accuracy}')

Accuracy: 0.3333333333333333


# Backpropagation
Introduce backward method that will carry out backpropagation.

1. We will want to remember the inputs to calculate the partial derivatives wrt weights during back propagation.

In [151]:
z = np.array([[ 1 , 2 , - 3 , - 4 ],
              [ 2 , - 7 , - 1 , 3 ],
              [ - 1 , 2 , 5 , - 1 ]])
drelu = np.zeros_like(z)
drelu[z>0] = 1
drelu

array([[1, 1, 0, 0],
       [1, 0, 0, 1],
       [0, 1, 1, 0]])

In [152]:
class Dense:
    """
    A layer of neurons.
    features: number of input features
    n_neurons: number of neurons in a layer
    """
    def __init__(self, features: int, n_neurons: int):
        #Initalize weights and biases
        self.weights = 0.01 * np.random.randn(features, n_neurons)
        self.biases = np.zeros((1, n_neurons))
    
    def forward(self, inputs: ndarray):
        """Calculate output values from inputs, weights and biases"""
        self.inputs = inputs
        self.output = inputs @ self.weights + self.biases
        
    def backward(self, dvalues:ndarray):
        """Calclates the gradient on parameters and values

        Args:
            dvalues (ndarray): passed in gradient from the next layer
        """
        self.dweights = self.inputs.T @ dvalues
        self.dbiases = np.sum(dvalues, axis=0, keepdims=True)
        self.dinputs = dvalues @ self.weights.T

In [153]:
class ActivationReLU:
    """
    Rectified Linear Units activation function
    """
    def forward(self, inputs: ndarray):
        self.inputs = inputs
        self.output = np.maximum(0, inputs)

    def backward(self, dvalues: ndarray):
        self.dinputs = dvalues.copy()
        self.dinputs[self.inputs <= 0] = 0

1. We turn numerical labels into OHE vectors.
2. Normalize the CCE gradient. Since optimizers sum all of the gradients related to each weight and bias before multiplying them by the learning rate, this sum will increase the more samples we have. Thus, we will ahve to adjust the learning rate according to each set of samples. Instead, we normalize the gradient by taking the mean, making their sum's magnitude invarient to the number of samples

In [154]:
class Loss:
    """
    Base loss class that calculatges the data and regularizes losses given model
    output and ground truth values.
    """
    def calculate(self, output:ndarray, y:ndarray) -> float:
        loss = np.mean(self.forward(output, y))
        
        return  loss
        
class CatCrossEntropy(Loss):
    def forward(self, y_pred, y_true):
        # num of samples in a batch
        samples = len(y_pred)
        
        # clip data on both sides
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)
        
        # calculate probabilities for target values
        # if categorical labels
        if len(y_true.shape) == 1:
            confs = y_pred_clipped[range(samples), y_true]
        
        # mask values - if one-hot encoded labels
        elif len(y_true.shape) == 2:
            confs = np.sum(y_pred_clipped * y_true, axis=1)
            
        neg_log_likelihoods = -np.log(confs)
        
        return neg_log_likelihoods
    
    def backward(self, dvalues: ndarray, y_true:ndarray):
        samples, labels = dvalues.shape
        # if sparse labels, convert to OHE vectors
        if len(y_true.shape) == 1:
            y_true = np.eye(labels)[y_true]
        
        # calclate gradient
        self.dinputs = -y_true / dvalues
        # noramlize gradient
        self.dinputs = self.dinputs / samples

## Derivative: Softmax activation function
The derivative to the softmax activation function can be calculated as:
$$\frac{\partial S_{i,j}}{\partial z_{i,k}} = S_{i,j}.(\delta_{j,k} - S_{i,k})$$ 
where the delta denotes the Kronecker function. Proof is omitted here.

### Code implementation

In [155]:
softmax_output = np.array([0.7, 0.1, 0.2]).reshape(-1,1)
softmax_output

# the left side of the equation (when distributed) is the Softmax's output multiplied by the Kronecker delta,
# which quals 1 when both inputs are equal ad 0 otherwise. When visualized as an array,
# we'll have an identity matrix

# help(np.diagflat)
first = np.diagflat(softmax_output)
print(first)

# the right part of the equation is the product of the softmax outputs, iterating over the
# j and k indices respectively

second = softmax_output @ softmax_output.T
print(second)

# finally, to get the derivative
print(first - second) 

[[0.7 0.  0. ]
 [0.  0.1 0. ]
 [0.  0.  0.2]]
[[0.49 0.07 0.14]
 [0.07 0.01 0.02]
 [0.14 0.02 0.04]]
[[ 0.21 -0.07 -0.14]
 [-0.07  0.09 -0.02]
 [-0.14 -0.02  0.16]]


In [156]:
class ActivationSoftmax:
    """
    Softmax activation function
    """
    def forward(self, inputs: ndarray):
        # get exp values whilst preventing overflow
        self.inputs = inputs
        exp_values = np.exp(inputs - np.max(inputs, axis=1, keepdims=True))
        
        #normalize for each sample
        probabs = exp_values / np.sum(exp_values, axis=1, keepdims=True)
        
        self.output = probabs
        
    def backward(self, dvalues: ndarray):
        # create unintialized array
        self.dinputs = np.empty_like(dvalues)
        # enumerate outputs and gradients
        for ix,(single_output, single_dvalues) in enumerate(zip(self.output, dvalues)):
        # flatten output array
            single_output = single_output.reshape(-1, 1)
            # calculate the Jacobian matrix of the output
            # j_matrix = np.diagflat(single_output) - (single_output @ single_output.T)
            j_matrix = np.diagflat(single_output) - np.dot(single_output, single_output.T)
            # calculate sample-wise gradient and add to array
            # self.dinputs[ix] = j_matrix @ single_dvalues
            self.dinputs[ix] = np.dot(j_matrix, single_dvalues)
  

## Combining Softmax and Cross-Entropy
So far, we have calculated the partial derivatives of the CCE and softmax. However, there is a way to speed things up (ew loops). The derivatives of both functions combine to solve a simple equation. Proof omitted here.

Applying the chain rule to calculate the partial derivative of the CCE w.r.t. the softmax inputs gives us:
$$\frac{\partial L_{i}}{\partial z_{i,k}} = \hat y_{i,k} - y_{i,k}$$

Isn't she pretty?

Note: to implement the solution, instead of performing the subtraction of the full arrays, we take advantage of the fact that y-true consists of OHE vectors. Thus, there is only a singular value of 1 in these vectors. This means we can index the prediction array with the sample number and its true value index, subtractging 1 from these. This requires discrete true labels and we force this in the code.


In [157]:
y_true = np.array([[ 1 , 0 , 0 ],[ 0 , 0 , 1 ],[ 0 , 1 , 0 ]])
print(np.argmax(y_true))
print(np.argmax(y_true, axis=1))

0
[0 2 1]


In [158]:
class SoftmaxCCE():
    """
    Calculates the parital derivative of the CCE w.r.t. the inputs of the softmax activation
    function.
    """
    def __init__(self):
        self.activation = ActivationSoftmax()
        self.loss = CatCrossEntropy()
        
    def forward(self, inputs: ndarray, y_true: ndarray):
        self.activation.forward(inputs)
        self.output = self.activation.output
        # calculate and return loss value
        return self.loss.calculate(self.output, y_true)
    
    def backward(self, dvalues, y_true):
        samples = len(dvalues)
        # if labels are OHE, turn to discrete
        if len(y_true.shape) == 2:
            y_true = np.argmax(y_true, axis=1)
        
        self.dinputs = dvalues.copy()
        # calculate gradient
        self.dinputs[range(samples), y_true] -= 1
        # normalize gradient
        self.dinputs = self.dinputs / samples

### Testing backpropogation

In [159]:
softmax_outputs
class_targets = np.array([0, 1, 1])

# solution awesome
softmax_loss = SoftmaxCCE()
softmax_loss.backward(softmax_outputs, class_targets)
dvalues_1 = softmax_loss.dinputs

# solution okay
activation = ActivationSoftmax()
activation.output = softmax_outputs
loss = CatCrossEntropy()
loss.backward(softmax_outputs, class_targets)
print(loss.dinputs)
activation.backward(loss.dinputs)
dvalues_2 = activation.dinputs

print(f'Gradients: combined loss and activation: {dvalues_1}')
print(f'Gradients: separate loss and activation: {dvalues_2}') 

[[-0.47619048 -0.         -0.        ]
 [-0.         -0.66666667 -0.        ]
 [-0.         -0.37037037 -0.        ]]
Gradients: combined loss and activation: [[-0.1         0.03333333  0.06666667]
 [ 0.03333333 -0.16666667  0.13333333]
 [ 0.00666667 -0.03333333  0.02666667]]
Gradients: separate loss and activation: [[-0.09999999  0.03333334  0.06666667]
 [ 0.03333334 -0.16666667  0.13333334]
 [ 0.00666667 -0.03333333  0.02666667]]


### Continued: Calculating loss 

In [160]:
# Since we added backward methods, we will have to reinitialze the layers
nnfs.init()
X, y = spiral_data(samples=100, classes=3)
# first layer with 3 neurons
dense_1 = Dense(2, 3)

# ReLU activation for first layer
activation_1 = ActivationReLU()

# second layer with 3 input feautures (input features in the second layer correspond to the outputs from the first layer)
# and 3 neurons
dense_2 = Dense(3, 3)

# softmax activagtion for second layer
loss_activation = SoftmaxCCE()

# forward pass through the network
dense_1.forward(X)
activation_1.forward(dense_1.output) #
dense_2.forward(activation_1.output)

In [161]:
# perform forward pass through the actrivation loss function
# takes the output of the second dense layer and returns loss
loss = loss_activation.forward(dense_2.output, y)
print(loss)

# backward pass
loss_activation.backward(loss_activation.output, y)
dense_2.backward(loss_activation.dinputs)
activation_1.backward(dense_2.dinputs)
dense_1.backward(activation_1.dinputs)

print(dense_1.dweights)
print('\n')
print(dense_1.dbiases)
print('\n')
print(dense_2.dweights)
print('\n')
print(dense_2.dbiases)

1.0986104
[[ 1.57663686e-04  7.83686628e-05  4.73244181e-05]
 [ 1.81610390e-04  1.10455085e-05 -3.30963376e-05]]


[[-3.60553531e-04  9.66117223e-05 -1.03671344e-04]]


[[ 5.44108079e-05  1.07411492e-04 -1.61822391e-04]
 [-4.07912230e-05 -7.16780778e-05  1.12469665e-04]
 [-5.30113066e-05  8.58172934e-05 -3.28059614e-05]]


[[-1.0732794e-05 -9.4590941e-06  2.0027626e-05]]


# Optimization

In [162]:
class StocGradDescent:
    # Initialize optimizer - set settings,
    # learning rate of 1. is default for this optimizer
    def __init__ ( self , learning_rate = 1.0 ):
        self.learning_rate = learning_rate
    # Update parameters
    def update_params ( self , layer ):
        layer.weights -= self.learning_rate * layer.dweights
        layer.biases -= self.learning_rate * layer.dbiases

In [168]:
nnfs.init()
# test
X, y = spiral_data(samples=100, classes=3)

# set up the neural network
dense_1 = Dense(2, 64)
activation_1 = ActivationReLU()
dense_2 = Dense(64, 3)
loss_activation = SoftmaxCCE()
optim = StocGradDescent()
# training over epochs
for epoch in range(10001):
    # perform forward pass
    dense_1.forward(X)
    activation_1.forward(dense_1.output)
    dense_2.forward(activation_1.output)
    loss = loss_activation.forward(dense_2.output, y)
    
    # calculate accuracy
    preds = np.argmax(loss_activation.output, axis=1)
    if len(y.shape) == 2:
        y = np.argmax(y, axis=1)
    accuracy = np.mean(preds == y)

    if not epoch % 100:
        print(f'epoch: {epoch}, accuracy: {accuracy:.3f}, loss: {loss:.3f}')
    # perform backward pass
    loss_activation.backward(loss_activation.output, y)
    dense_2.backward(loss_activation.dinputs)
    activation_1.backward(dense_2.dinputs)
    dense_1.backward(activation_1.dinputs)

    # update weights and biases
    optim.update_params(dense_1)
    optim.update_params(dense_2)

epoch: 0, accuracy: 0.360, loss: 1.099
epoch: 100, accuracy: 0.400, loss: 1.087
epoch: 200, accuracy: 0.417, loss: 1.077
epoch: 300, accuracy: 0.413, loss: 1.076
epoch: 400, accuracy: 0.400, loss: 1.074
epoch: 500, accuracy: 0.400, loss: 1.071
epoch: 600, accuracy: 0.417, loss: 1.067
epoch: 700, accuracy: 0.437, loss: 1.062
epoch: 800, accuracy: 0.457, loss: 1.055
epoch: 900, accuracy: 0.410, loss: 1.062
epoch: 1000, accuracy: 0.403, loss: 1.059
epoch: 1100, accuracy: 0.403, loss: 1.057
epoch: 1200, accuracy: 0.397, loss: 1.064
epoch: 1300, accuracy: 0.433, loss: 1.051
epoch: 1400, accuracy: 0.390, loss: 1.099
epoch: 1500, accuracy: 0.477, loss: 1.044
epoch: 1600, accuracy: 0.407, loss: 1.060
epoch: 1700, accuracy: 0.410, loss: 1.037
epoch: 1800, accuracy: 0.400, loss: 1.036
epoch: 1900, accuracy: 0.403, loss: 1.039
epoch: 2000, accuracy: 0.433, loss: 1.046
epoch: 2100, accuracy: 0.500, loss: 1.014
epoch: 2200, accuracy: 0.470, loss: 1.042
epoch: 2300, accuracy: 0.470, loss: 1.013
epoc

## Learning Rate Decay
The idea of a learning rate decay is to start with a large learning rate and then decrease it during trainig. We will use a 1/t decaying or exponential decaying rate. Initially, the lr drops fast, but the change in lr lowers with each step.

Note: the 1 + learning rate ensures that the result is always a fraction of the starting learning rate i.e. will be always less than or equal to 1.

In [164]:
init_learning_rate = 1
decay_rate = 0.1
step = 1

learning_rate = init_learning_rate * (1 / (1 + decay_rate * step))

print(learning_rate)

for step in range(20):
    learning_rate = init_learning_rate * (1 / (1 + decay_rate * step))
    print(f'{learning_rate:.3f}')    

0.9090909090909091
1.000
0.909
0.833
0.769
0.714
0.667
0.625
0.588
0.556
0.526
0.500
0.476
0.455
0.435
0.417
0.400
0.385
0.370
0.357
0.345


### Updated Optimizer

In [169]:
class StocGradDescent:
    # Initialize optimizer - set settings,
    # learning rate of 1. is default for this optimizer
    def __init__ ( self , learning_rate: float=1.0, decay: float=0.):
        self.learning_rate = learning_rate
        self.current_lr = learning_rate
        self.decay = decay
        self.iterations = 0
        # if self.decay:
        #     self.current_lr = self.learning_rate * \
        #         (1. / (1. + self.decay * self.iterations))
    # Update parameters
    def update_params(self, layer:Dense):
        layer.weights -= self.current_lr * layer.dweights
        layer.biases -= self.current_lr * layer.dbiases
        
    def pre_update_params(self):
        """
        Updates self.current_lr if decay rate is non-zero.
        Call once before any paramter updates.
        """
        if self.decay:
            self.current_lr = self.learning_rate * \
                (1. / (1. + self.decay * self.iterations))
            
    def update_iter(self):
        """Tracks iterations by updating self.iterations
        """
        self.iterations += 1
    

In [170]:
nnfs.init()
# test
X, y = spiral_data(samples=100, classes=3)

# set up the neural network
dense_1 = Dense(2, 64)
activation_1 = ActivationReLU()
dense_2 = Dense(64, 3)
loss_activation = SoftmaxCCE()
optim = StocGradDescent(decay=1e-3)
# training over epochs
for epoch in range(10001):
    # perform forward pass
    dense_1.forward(X)
    activation_1.forward(dense_1.output)
    dense_2.forward(activation_1.output)
    loss = loss_activation.forward(dense_2.output, y)
    
    # calculate accuracy
    preds = np.argmax(loss_activation.output, axis=1)
    if len(y.shape) == 2:
        y = np.argmax(y, axis=1)
    accuracy = np.mean(preds == y)

    if not epoch % 1000:
        print(f'epoch: {epoch}, accuracy: {accuracy:.3f}, \
              loss: {loss:.3f}, lr: {optim.current_lr:.3f}')
    # perform backward pass
    loss_activation.backward(loss_activation.output, y)
    dense_2.backward(loss_activation.dinputs)
    activation_1.backward(dense_2.dinputs)
    dense_1.backward(activation_1.dinputs)

    # update weights and biases
    optim.pre_update_params()
    optim.update_params(dense_1)
    optim.update_params(dense_2)
    optim.update_iter()

epoch: 0, accuracy: 0.360,               loss: 1.099, lr: 1.000
epoch: 1000, accuracy: 0.443,               loss: 1.063, lr: 0.500
epoch: 2000, accuracy: 0.463,               loss: 0.999, lr: 0.333
epoch: 3000, accuracy: 0.490,               loss: 0.990, lr: 0.250
epoch: 4000, accuracy: 0.533,               loss: 0.947, lr: 0.200
epoch: 5000, accuracy: 0.577,               loss: 0.902, lr: 0.167
epoch: 6000, accuracy: 0.630,               loss: 0.860, lr: 0.143
epoch: 7000, accuracy: 0.673,               loss: 0.824, lr: 0.125
epoch: 8000, accuracy: 0.643,               loss: 0.799, lr: 0.111
epoch: 9000, accuracy: 0.640,               loss: 0.783, lr: 0.100
epoch: 10000, accuracy: 0.670,               loss: 0.771, lr: 0.091


## Momentum
Momentum creates a rolling average of gradients over some number of updates and uses this average with the uniquw gradient at each step. With momentum, a model is more likely to pass through local minimums, further decreasing loss.

We set a parameter between 0 and 1, representing the fraction of the previous parameter update to retain, and subtracting our actual gradient multipplied by the learning rate, from it. The update contains a portion of the gradient from the preceding steps as our momentum and only a portion of the current gradient. Together, these portions form the actual change to our parameters.

The bigger the role that momentum takes in the update, the slower the update can change the direction.

weight_updates = self.momentum * layer.weight_momentums - \
(self.current_learning_rate * layer.dweights) 

The hyper-parameter self.momentum is chosen at the start and the layer.weight_momentums starts as all zeros but update during training.

### SGD Optimizer with momentum

In [175]:
class StocGradDescent:
    # Initialize optimizer - set settings,
    # learning rate of 1. is default for this optimizer
    def __init__ ( self , learning_rate: float=1.0, decay: float=0.,
                  momentum=0.):
        self.learning_rate = learning_rate
        self.current_lr = learning_rate
        self.decay = decay
        self.iterations = 0
        self.momentum = momentum
        # if self.decay:
        #     self.current_lr = self.learning_rate * \
        #         (1. / (1. + self.decay * self.iterations))
    # Update parameters
    def update_params(self, layer:Dense):
        # if using momentum
        if self.momentum:
            # create momentum arrays for layer if layer
            # does not contain it.
            if not hasattr(layer, 'weight_momentums'):
                layer.weight_momentums = np.zeros_like(layer.weights)
                layer.bias_momentums = np.zeros_like(layer.biases)
            
            # take prev updates times retain factor and update with
            # current gradients
            weight_updates = self.momentum * layer.weight_momentums + \
                self.current_lr * layer.dweights
            layer.weight_momentums = weight_updates        
            
            bias_updates = self.momentum * layer.bias_momentums + \
                self.current_lr * layer.dbiases
            layer.bias_momentums = bias_updates        
        
        else:   
            weight_updates = self.current_lr * layer.dweights
            bias_updates = self.current_lr * layer.dbiases
        
        # update weights and biases
        layer.weights -= weight_updates
        layer.biases -= bias_updates
        
    def pre_update_params(self):
        """
        Updates self.current_lr if decay rate is non-zero.
        Call once before any paramter updates.
        """
        if self.decay:
            self.current_lr = self.learning_rate * \
                (1. / (1. + self.decay * self.iterations))
            
    def update_iter(self):
        """Tracks iterations by updating self.iterations
        """
        self.iterations += 1

In [180]:
nnfs.init()
X, y = spiral_data(samples=100, classes=3)

# set up the neural network
dense_1 = Dense(2, 64)
activation_1 = ActivationReLU()
dense_2 = Dense(64, 3)
loss_activation = SoftmaxCCE()
optim = StocGradDescent(decay=1e-3, momentum=0.9)
# training over epochs
for epoch in range(10001):
    # perform forward pass
    dense_1.forward(X)
    activation_1.forward(dense_1.output)
    dense_2.forward(activation_1.output)
    loss = loss_activation.forward(dense_2.output, y)
    
    # calculate accuracy
    preds = np.argmax(loss_activation.output, axis=1)
    if len(y.shape) == 2:
        y = np.argmax(y, axis=1)
    accuracy = np.mean(preds == y)

    if not epoch % 100:
        print(f'epoch: {epoch}, accuracy: {accuracy:.3f}, \
              loss: {loss:.3f}, lr: {optim.current_lr:.3f}')
    # perform backward pass
    loss_activation.backward(loss_activation.output, y)
    dense_2.backward(loss_activation.dinputs)
    activation_1.backward(dense_2.dinputs)
    dense_1.backward(activation_1.dinputs)

    # update weights and biases
    optim.pre_update_params()
    optim.update_params(dense_1)
    optim.update_params(dense_2)
    optim.update_iter()

epoch: 0, accuracy: 0.360,               loss: 1.099, lr: 1.000
epoch: 100, accuracy: 0.443,               loss: 1.053, lr: 0.910
epoch: 200, accuracy: 0.543,               loss: 0.972, lr: 0.834
epoch: 300, accuracy: 0.657,               loss: 0.763, lr: 0.770
epoch: 400, accuracy: 0.727,               loss: 0.676, lr: 0.715
epoch: 500, accuracy: 0.787,               loss: 0.571, lr: 0.667
epoch: 600, accuracy: 0.807,               loss: 0.500, lr: 0.625
epoch: 700, accuracy: 0.823,               loss: 0.434, lr: 0.589
epoch: 800, accuracy: 0.830,               loss: 0.435, lr: 0.556
epoch: 900, accuracy: 0.840,               loss: 0.412, lr: 0.527
epoch: 1000, accuracy: 0.850,               loss: 0.388, lr: 0.500
epoch: 1100, accuracy: 0.873,               loss: 0.346, lr: 0.476
epoch: 1200, accuracy: 0.857,               loss: 0.369, lr: 0.455
epoch: 1300, accuracy: 0.873,               loss: 0.327, lr: 0.435
epoch: 1400, accuracy: 0.877,               loss: 0.321, lr: 0.417
epoch: 

## AdaGrad
AdaGrad or Adaptive Gradient, institues a per-parameter learning rate rahter than a globally-shared rate. AdaGrad provides a way to normalize parameter updates by keeping a history of previous updates - the bigger the sum of the updates, positive or negative, the smaller updates are made further in training.

cache += parm_gradient ** 2
parm_updates = learning_rate * parm_gradient / (sqrt(cache) + eps)

The cache holds a history of the squared gradients. The parm__updates follows a vanill SGD but is then divided by the square root of the cache plus some epsilon value. Note: the division operation with a constantaly rising cache might cause the learning to stall as updates become smaller over time (monotonic nature of updates). Hence, this optimizer is not widely used, except for specific applications.

Epsilon is a hyperparameter preventing division by 0. 

Notice also that we sum the squared value and then square root later. The resulting cache value will grow slower, taking care of negative numbers

In [182]:
class AdaGrad:
    # Initialize optimizer - set settings,
    # learning rate of 1. is default for this optimizer
    def __init__ ( self , learning_rate: float=1.0, decay: float=0.,
                  epsilon=1e-7):
        self.learning_rate = learning_rate
        self.current_lr = learning_rate
        self.decay = decay
        self.iterations = 0
        self.epsilon = epsilon
        # if self.decay:
        #     self.current_lr = self.learning_rate * \
        #         (1. / (1. + self.decay * self.iterations))
    # Update parameters
    def update_params(self, layer:Dense):
        # create cache arrays for layer if layer
        # does not contain it.
        if not hasattr(layer, 'weight_cache'):
            layer.weight_cache = np.zeros_like(layer.weights)
            layer.bias_cache = np.zeros_like(layer.biases)
        
        # update cachec with squared gradients
        layer.weight_cache += layer.dweights ** 2
        layer.bias_cache += layer.dbiases ** 2
        
        layer.weights -= self.current_lr * layer.dweights / \
            (np.sqrt(layer.weight_cache) + self.epsilon)
        layer.biases -= self.current_lr * layer.dbiases / \
            (np.sqrt(layer.bias_cache) + self.epsilon)
        
    def pre_update_params(self):
        """
        Updates self.current_lr if decay rate is non-zero.
        Call once before any paramter updates.
        """
        if self.decay:
            self.current_lr = self.learning_rate * \
                (1. / (1. + self.decay * self.iterations))
            
    def update_iter(self):
        """Tracks iterations by updating self.iterations
        """
        self.iterations += 1

In [185]:
nnfs.init()
X, y = spiral_data(samples=100, classes=3)

# set up the neural network
dense_1 = Dense(2, 64)
activation_1 = ActivationReLU()
dense_2 = Dense(64, 3)
loss_activation = SoftmaxCCE()
optim = AdaGrad(decay=1e-4)
# training over epochs
for epoch in range(10001):
    # perform forward pass
    dense_1.forward(X)
    activation_1.forward(dense_1.output)
    dense_2.forward(activation_1.output)
    loss = loss_activation.forward(dense_2.output, y)
    
    # calculate accuracy
    preds = np.argmax(loss_activation.output, axis=1)
    if len(y.shape) == 2:
        y = np.argmax(y, axis=1)
    accuracy = np.mean(preds == y)

    if not epoch % 100:
        print(f'epoch: {epoch}, accuracy: {accuracy:.3f}, \
              loss: {loss:.3f}, lr: {optim.current_lr:.3f}')
    # perform backward pass
    loss_activation.backward(loss_activation.output, y)
    dense_2.backward(loss_activation.dinputs)
    activation_1.backward(dense_2.dinputs)
    dense_1.backward(activation_1.dinputs)

    # update weights and biases
    optim.pre_update_params()
    optim.update_params(dense_1)
    optim.update_params(dense_2)
    optim.update_iter()

epoch: 0, accuracy: 0.360,               loss: 1.099, lr: 1.000
epoch: 100, accuracy: 0.460,               loss: 1.010, lr: 0.990
epoch: 200, accuracy: 0.527,               loss: 0.937, lr: 0.980
epoch: 300, accuracy: 0.607,               loss: 0.872, lr: 0.971
epoch: 400, accuracy: 0.617,               loss: 0.825, lr: 0.962
epoch: 500, accuracy: 0.633,               loss: 0.784, lr: 0.952
epoch: 600, accuracy: 0.630,               loss: 0.768, lr: 0.943
epoch: 700, accuracy: 0.647,               loss: 0.741, lr: 0.935
epoch: 800, accuracy: 0.673,               loss: 0.727, lr: 0.926
epoch: 900, accuracy: 0.690,               loss: 0.689, lr: 0.918
epoch: 1000, accuracy: 0.687,               loss: 0.676, lr: 0.909
epoch: 1100, accuracy: 0.693,               loss: 0.657, lr: 0.901
epoch: 1200, accuracy: 0.703,               loss: 0.644, lr: 0.893
epoch: 1300, accuracy: 0.713,               loss: 0.631, lr: 0.885
epoch: 1400, accuracy: 0.710,               loss: 0.616, lr: 0.877
epoch: 

## RMSProp
Similar to AdaGradm, Root Mean Sqared Propogation caculates an adaptive learning rate per parameter.

cache = rho * cache + (1 - rho) * gradient ** 2